# **Resident Manta Activity**
## Outputs: 
> ### 1) Callable dataset: manta_daily_summary.xlsx (Step 2)
> ### 2) Time Series of Daily Total Detections (Step 3)
> ### 3) Heatmaps (Step 4): 
>> #### 3.1) Monthly by Day per individual Manta
>> #### 3.2) Yearly by Month per individual Manta
>> #### 3.3) Yearly by Month averaged between Mantas
> ### 4) Statistics for Time-of-Day Significance (Step 5)
> ### 5) Statistics for Agreement Between Mantas (Step 6)
> ### 6) Interpretation of Agreement and TOD (Step 7)

___________________________

## Step 0: Importing and Notebook Set-Up

In [1]:
#########################################################################################################
# This block of code is importing all the necessary packages and functions needed for the entire
# notebook, no further imports are made scrolling down. It also loads the necessary excel file, which
# is a truncated version of the Manta data in which all data has been removed except the necessary
# columns, and only for the two mantas of interest (the full-time resident manta, and the manta who
# returned as a visitor over multiple years). It creates a folder for all plots to be placed into inside
# edstem, and creates labels for the two mantas to be used on plots instead of their transmitter IDs. 
#########################################################################################################  

import os
import numpy as np
import pandas as pd  
import matplotlib.pyplot as plt 
from matplotlib.dates import DateFormatter #transforms dates to aesthetic formatting
import seaborn as sns #library inside matplotlib for more advanced plotting functions
from scipy.stats import f_oneway, pearsonr, spearmanr, chi2_contingency #statistical functions for use at end of notebook

base_dir = "ResidentMantas_Inputs"
file_path = os.path.join("ResidentMantas_Inputs", "Resident_Data.xlsx")

output_dir = "ResidentMantas_Outputs" #folder for plots/sheets to save to

folders = {
    "sheets": os.path.join(output_dir, "sheets"),
    "figures": os.path.join(output_dir, "figures"),
}

for f in folders.values():
    os.makedirs(f, exist_ok=True) #creates folders, prevents re-creation if already exists

#Creating sub-folders for different plot types 
timeseries_folder = os.path.join(folders["figures"], "Timeseries")
os.makedirs(timeseries_folder, exist_ok=True)

heatmaps_folder = os.path.join(folders["figures"], "Heatmaps")
os.makedirs(heatmaps_folder, exist_ok=True)

sns.set(style="whitegrid") #tells seaborn to use whitegrid style, sets standard for notebook

transmitter_labels = {
    "A69-1601-32687": "Return-Visitor Manta",
    "A69-1601-32688": "Resident Manta"
}

## Step 1: Reading Data and Organizing

In [2]:
df = pd.read_excel(file_path) #reading given excel file into pandas

df['detection_datetime'] = pd.to_datetime(df['detection_datetime']) #getting proper datetime format, allows extraction below

# Extracting specific time periods from datetime
df['date'] = df['detection_datetime'].dt.date 
df['datetime_hour'] = df['detection_datetime'].dt.floor('H') #rounding down to nearest hour, allows grouping within hours 
df['hour'] = df['detection_datetime'].dt.hour #extracting hour

# Creating time of day bins for Step 5
tod_bins = [0, 6, 12, 18, 24] #hour cut-off points 
tod_labels = ['Night (0-6)', 'Morning (6-12)', 'Afternoon (12-18)', 'Evening (18-24)']

# Counting raw hourly counts per manta (i.e., including counts under 2)
hourly_raw = (
    df.groupby(['transmitter_id', 'datetime_hour'], as_index=False) 
      .size() #gets count in hour per manta
      .rename(columns={'size': 'detections_in_hour'}) #creates column of counts
)

# Applying >=2 detections/hour rule to above
hourly_valid = hourly_raw[hourly_raw['detections_in_hour'] >= 2].copy() #inside inner parenthesis, pandas giving boolean if counts >=2, outer parenthesis only keeping True
hourly_valid['date'] = pd.to_datetime(hourly_valid['datetime_hour']).dt.date #getting date for valid counts
hourly_valid['hour'] = pd.to_datetime(hourly_valid['datetime_hour']).dt.hour #getting hour for valid counts

/tmp/ipykernel_56/3872296926.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['datetime_hour'] = df['detection_datetime'].dt.floor('H') #rounding down to nearest hour, allows grouping within hours


## Step 2: Build Callable Daily Dataset for Aggregation with Environmental Variables
> #### 1) Total Detections
> #### 2) Valid Hours per Day 
> #### 3) Avg Detections per Hour per Day

In [3]:
#########################################################################################################
# This block of code is creating a callable dataset to be used in other notebooks of this workspace.
# It is creating an excel file that contains a pandas datetime column (transformed), total detections
# per manta per day, valid hours per day per manta, and average detections per hour per day per manta.
#########################################################################################################

daily_totals = (hourly_valid.groupby(['transmitter_id', 'date'], as_index=False) #manta count by hour and data
                .agg(total_detections=('detections_in_hour', 'sum'), #getting summary for groups, total detections and valid hours
                     valid_hours=('datetime_hour', 'nunique'))
)

daily_totals['avg_detections_per_hour'] = (
    daily_totals['total_detections'] / daily_totals['valid_hours'] #creating column for valid hours in a day
)

daily_totals['date_dt'] = pd.to_datetime(daily_totals['date']) #creating pandas datetime object to be used for plotting in new column, leaving original date column

# Save callable data to Excel File
daily_summary_path = os.path.join(folders["sheets"], "manta_daily_summary.xlsx")
daily_totals.to_excel(daily_summary_path, sheet_name='daily_summary_filtered', index=False)

## Step 3: Timeseries — Total Detections per Day 

### Step 3.1: Basic Timeseries Detections/Day


In [4]:
#########################################################################################################
# This block of code uses matplotlib and seaborn to make the more-basic time series figure that shows
# two lines (one per manta) of detections per day over the entire study period. 
#########################################################################################################

daily_plot_df = daily_totals.copy() #making a copy of the dataframe so that the original is not changed
daily_plot_df['Manta'] = daily_plot_df['transmitter_id'].map(
    transmitter_labels #creating column of previously-made dictionary in step 0
).fillna(daily_plot_df['transmitter_id']) #not really necessary since I know the input data was filtered to only these two IDs, but would stop another ID from becoming NaN

#setting up and creating figure 
plt.figure(figsize=(16, 8))

sns.lineplot( #using seaborn 
    data=daily_plot_df,
    x="date_dt",
    y="total_detections",
    hue="Manta", #asking for one line per manta, adds legend
    lw=1.5
) #creating line plot 

plt.title("Daily detections per Manta")
plt.xlabel("Date")
plt.ylabel("Detections per day")
plt.xticks(rotation=45)
plt.tight_layout()
ts_path = os.path.join(timeseries_folder, "Basic_resident_timeseries.png") #placing in folder created in step 0
plt.savefig(ts_path, dpi=300)
plt.close()

### Step 3.1: Daily + Monthly Timeseries Detections/Day (dual-axes)

In [5]:
#########################################################################################################
# This block of code make a similar plot to the one above, but it overlays a more-transparent daily
# count with an opaque monthly count (dual y-axes) to smooth the lines a bit. I couldn't figure out
# how to have one legend using seaborn with two y-axes (there were two overlapping) so this plot is
# not made with seaborn, but is made with matplotlib. 
#########################################################################################################

monthly_totals = (
    daily_totals.groupby(['transmitter_id', daily_totals['date_dt'].dt.to_period('M')]) #using pandas to group by month
    .agg(total_monthly=('total_detections', 'sum')) #monthly detection total counts
    .reset_index() #stopping indexing from groupby -- doesn't work without this
)
monthly_totals['month_start'] = monthly_totals['date_dt'].dt.to_timestamp() #takes period created by pandas and converts into datetime by first day of each month

# --- Pivot to align resident vs return-visitor for monthly correlation ---
aligned_monthly = monthly_totals.pivot(index='month_start', columns='transmitter_id', values='total_monthly')
aligned_monthly = aligned_monthly.rename(columns=transmitter_labels).dropna(how='any')

# --- Correlation between mantas (monthly activity trends) ---
r, p = pearsonr(aligned_monthly['Resident Manta'], aligned_monthly['Return-Visitor Manta'])
print(f"\nMonthly correlation between mantas:")
print(f"  Pearson r = {r:.3f} (p = {p:.3f})")
if p < 0.05:
    print("  → Their monthly detection trends are significantly correlated.")
else:
    print("  → Their monthly detection trends are not significantly correlated.")

#Have to set up daily count as above because this is dual-axes. Probably could have combined this block
#and the block above, but kept seperate for ease of reading and so that only one could be run if wanted
#Three lines below copying what was done above.

daily_plot_df = daily_totals.copy()
daily_plot_df['manta_label'] = daily_plot_df['transmitter_id'].map(transmitter_labels).fillna(daily_plot_df['transmitter_id'])
monthly_totals['manta_label'] = monthly_totals['transmitter_id'].map(transmitter_labels).fillna(monthly_totals['transmitter_id'])

#Set color palette for both series, didn't have to do above because seaborn does it inside the 'hue'
#part. Making the daily and monthly color per manta the same (i.e., one color per manta) by creating
#the color_map

manta_labels = daily_plot_df['manta_label'].unique() #getting number of unique mantas -- could have just written 2 in line below, but this code could now be used for a larger dataset
palette = sns.color_palette(n_colors=len(manta_labels)) #using seaborn coloring
color_map = dict(zip(manta_labels, palette))

#Creating matplot lib dual-axis figure 
fig, ax1 = plt.subplots(figsize=(16, 8))
ax2 = ax1.twinx()  # secondary y-axis

for manta in manta_labels: #loops through each manta (label), as identified above
    sub = daily_plot_df[daily_plot_df['manta_label'] == manta] #getting daily data for each given manta
    ax1.plot(
        sub['date_dt'],
        sub['total_detections'],
        color=color_map[manta],
        alpha=0.5,
        linewidth=1,
        label=f"{manta} (daily)"
    ) #plotting daily detections as transparent thinner line


for manta in manta_labels:
    sub = monthly_totals[monthly_totals['manta_label'] == manta]
    ax2.plot(
        sub['month_start'],
        sub['total_monthly'],
        color=color_map[manta],
        linewidth=2,
        label=f"{manta} (monthly)"
    ) #plotting monthly counts 

ax1.set_title("Daily Detections per Manta (Overlayed with Monthly Detections)")
ax1.set_xlabel("Date")
ax1.set_ylabel("Daily Detections")
ax2.set_ylabel("Monthly Detections")

#Combining legends
lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper right', title='Manta')

ax1.tick_params(axis='x', rotation=45)
ax1.grid(False) #removed gridlines because the two axes overlapped strangely 
fig.tight_layout()

# Save figure
dual_ts_path = os.path.join(timeseries_folder, "DailyMonthly_resident_dual_timeseries.png")
plt.savefig(dual_ts_path, dpi=300)
plt.close()


Monthly correlation between mantas:
  Pearson r = 0.385 (p = 0.141)
  → Their monthly detection trends are not significantly correlated.


## Step 4: Heatmaps

### Step 4.1: Daily per Month per Manta

In [6]:
#########################################################################################################
# This block creates a heatmap of daily detections for a given month and year (in the first two lines of
# code) If that month/year for a manta is one of zero detections, the heatmap will still be created and
# will be all one color (because no data). 
#########################################################################################################

heatmap_year = 2012   # can be altered to any year 2011-2013
heatmap_month = 9    # can be altered to any month

# Global min and per-manta max's for consistent colour scale -- initially had one global max but did not make sense with vastly different behaviors 
heatmap_min   = 0 
#Max for resident plots
resident_filter = daily_totals["transmitter_id"] == list(transmitter_labels.keys())[1]
resident_max = daily_totals.loc[resident_filter, "total_detections"].max()
#Max for return-visitor plots
return_filter   = daily_totals["transmitter_id"] == list(transmitter_labels.keys())[0]
return_max   = daily_totals.loc[return_filter, "total_detections"].max()


#Creating day index for given month/year
month_start = pd.Timestamp(year=heatmap_year, month=heatmap_month, day=1) #takes inputs from above and creating pandas timestamp
month_end = (month_start + pd.offsets.MonthEnd(1)) #.MonthEnd goes to the end of the given month (knows month lengths, leap years, etc.)
given_days = pd.date_range(month_start, month_end, freq='D') #using dates from last two lines and creating date range

#Plotting 1-column heatmaps
for manta_id in daily_totals['transmitter_id'].unique(): #looping through each manta
    label = transmitter_labels.get(manta_id, manta_id) #getting label from dictionary, would return transmitter ID if no matching key

    sub = daily_totals[ #selecting data for ....
        (daily_totals['transmitter_id'] == manta_id) & # given manta, 
        (daily_totals['date_dt'].dt.year == heatmap_year) & # given year, 
        (daily_totals['date_dt'].dt.month == heatmap_month) # given month.
    ].copy() 

    #Indexing to date instead of row number (because not all days have detections) and filling empty days with 0
    day_series = sub.set_index('date_dt')['total_detections'].reindex(given_days, fill_value=0)

    #Builds a dataframe with one column where index = day of month and column value is count so that seaborn can use it
    day_df = pd.DataFrame({'Total detections': day_series.values}, index=given_days.day)
    
    if "Resident" in label:
        heatmap_max = resident_max
    elif "Return" in label:
        heatmap_max = return_max
    else:
        heatmap_max = daily_totals["total_detections"].max()

    plt.figure(figsize=(4, 8))
    sns.heatmap(
        day_df,
        cmap='viridis',
        vmin= heatmap_min,
        vmax= heatmap_max,
        cbar_kws={'label': 'Daily total detections'},
        yticklabels=True
    )
    plt.title(f"Daily Counts — {label}\n{heatmap_year}-{heatmap_month:02d}") #creates title with manta name and given month/year
    plt.xlabel("")  #removing x-axis label
    plt.ylabel("Day of month")
    plt.tight_layout()
    hm1_path = os.path.join(heatmaps_folder, f"DayxMonth_heatmap_{label.replace(' ','_')}_{heatmap_year}_{heatmap_month:02d}.png") #creating title for output that is specific to inputs.png")
    plt.savefig(hm1_path, dpi=300)
    plt.close()

### Step 4.2: Monthly per Manta (2011-2013)

In [7]:
# This block of code is setting up abbreviations for month to use in Step 4.2
# ----------------------
MONTH_ABBR = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'] #indexing months (i.e., 0=Jan)
MONTH_TO_ABBR = {m: MONTH_ABBR[m-1] for m in range(1,13)} #changing from index starting at 0 (i.e., 1=Jan)

In [8]:
#########################################################################################################
# This block creates a heatmap of monthly detections across 2011-2013 for each manta (resident and
# repeat returner). 
#########################################################################################################

daily_totals['year'] = daily_totals['date_dt'].dt.year #getting year and month (below) 
daily_totals['month'] = daily_totals['date_dt'].dt.month

for manta_id in daily_totals['transmitter_id'].unique(): #looping through each manta as above
    label = transmitter_labels.get(manta_id, manta_id)

    sub = daily_totals[daily_totals['transmitter_id'] == manta_id].copy() #selecting data for given manta

    monthly_sum = (sub.groupby(['year', 'month'])['total_detections'] #grouping detections by month and year
                     .sum()
                     .reset_index())

    #Sets up a grid of years and months for heatmap
    years = sorted(monthly_sum['year'].unique())
    months = list(range(1, 12+1))

    #converting df into pivot table for heatmap (in code above just left as df because one column)
    pivot = monthly_sum.pivot(index='year', columns='month', values='total_detections')
    pivot = pivot.reindex(index=years, columns=months).fillna(0) #shows no-detection months
    pivot.columns = [MONTH_TO_ABBR[m] for m in pivot.columns] 

    plt.figure(figsize=(12, 4)) 
    sns.heatmap(
        pivot,
        cmap='viridis',
        cbar_kws={'label':'Monthly total detections'},
        annot=False
    )
    plt.title(f"Monthly Totals — {label}")
    plt.xlabel("Month")
    plt.ylabel("Year")
    plt.tight_layout()

    hm2_path = os.path.join(heatmaps_folder, f"MonthxYear_heatmap_{label.replace(' ','_')}.png") #creating title for output that is specific to inputs.png")
    plt.savefig(hm2_path, dpi=300)
    plt.close()

### Step 4.3: Monthly per Year; Manta's averaged

In [9]:
#########################################################################################################
# This block creates a heatmap of monthly detections across 2011-2013 for mantas averaged (resident and
# repeat returner). Same as cell above but gives a look at manta activity more generally rather
# than looking at individuals.  
#########################################################################################################

monthly_all = (daily_totals.groupby(['transmitter_id', 'year', 'month'])['total_detections']
    .sum() #summing total detections for each manta by month
    .reset_index()
)

monthly_avg = (monthly_all.groupby(['year', 'month'])['total_detections']
    .mean() #averaging across mantas
    .reset_index()
)

#as above, changing data into pivot table for use in seaborn heatmap format
years = sorted(monthly_avg['year'].unique())
months = list(range(1, 13))
pivot_avg = (
    monthly_avg
    .pivot(index='year', columns='month', values='total_detections')
    .reindex(index=years, columns=months)
    .fillna(0)
)
pivot_avg.columns = [MONTH_TO_ABBR[m] for m in pivot_avg.columns]


plt.figure(figsize=(12, 4))
sns.heatmap(
    pivot_avg,
    cmap='viridis',
    cbar_kws={'label':'Avg Monthly Total Detections'},
    annot=False
)
plt.title("Monthly totals — mean across mantas")
plt.xlabel("Month")
plt.ylabel("Year")
plt.tight_layout()
hm3_path = os.path.join(heatmaps_folder, "MonthxYear_avg_heatmap.png")
plt.savefig(hm3_path, dpi=300)
plt.close()

## Step 5: Statistics for Time-of-Day

In [10]:
#########################################################################################################
# Even though we don't have other parameters in hourly counts, we wanted to see if there was anything
# interesting in manta detections given the time of day (as indicated in literature)! This block of
# code is seeing if time-of-day is significant in amount of  detections per hour / activity for each
# manta and mantas averaged by doing a one-way ANOVA test. It then runs brief agreement tests to see
# if the TOD preferences agree. AI used for help in this block for debugging and stat-package jargon assistance.
#########################################################################################################

def oneway_anova_by_bin(df_in, label="(all)"): #creating function to run one-way anova test with input df

    #Seperating data into numpy arrays by time of day category and getting total detections/day for each array
    groups = [grp['bin_total'].values for _, grp in df_in.groupby('time_of_day', observed=True)]
    
    #Skips ANOVA test if there aren't detections across time of day groups
    non_empty = [g for g in groups if len(g) > 0] #g = groups, each is a numpy array of total detections for a TOD bin
    if len(non_empty) < 2:
        return {'Manta': label, 'F-stat': np.nan, 'p-val': np.nan, 'eta^2': np.nan}

    #Run one-way ANOVA, stat = F statistic, p = p-value
    stat, p = f_oneway(*non_empty) #f_oneway multiple arguments, * is unpacking non_empty list

    #Quantifying how much variation can be explained by time-of-day by Eta squared (effect size matrix)
    #Given by variation between tod groups / total variation, higher values = larger effect

    overall = df_in['bin_total'].mean() #calculating overall mean of detections across all TOD bins
    ss_between = sum(len(grp) * ((grp['bin_total'].mean() - overall) ** 2)
                     for _, grp in df_in.groupby('time_of_day', observed=True)) #for top of fraction, getting sum of squares/variation between bins
    ss_total = ((df_in['bin_total'] - overall) ** 2).sum() #for bottom of fraction, total variability
    eta_sq = ss_between / ss_total if ss_total > 0 else np.nan #calculating effect with condition in case of denominator 0


    #Computing mean detections per time-of-day bin given in pandas series
    bin_means = (df_in.groupby('time_of_day', observed=True)['bin_total']
                     .mean()
                     .reindex(tod_labels) #ordering bins morning --> night
                     .round(2)) #rounding decimal points 

    #Collecting outputs into singular dictionary with Manta name, F-statistic, p-value, effect size, and mean detections per time of day bin
    anova = {'Manta': label, 'F-stat': stat, 'p-val': p, 'eta^2': eta_sq}
    for bin, avg in bin_means.items(): #indexing above series like dictionary
        anova[f"mean_{bin}"] = avg #for each TOD bin and avg detections add entry to anova dict
    return anova #returns dictionary of statistics

#End of function
######################################################################################
#Assigns time of day categories by hour to dataset (using bins and labels from Step 0)
hourly_valid['time_of_day'] = pd.cut(
    hourly_valid['hour'],
    bins=tod_bins,
    labels=tod_labels,
    right=False, #upper-bound automatically included -- this is changing that (i.e., 6 is morning not night)
    include_lowest=True #includes 0 into night
)

#Detections per manta per day per time of day bin 
tod_day = (hourly_valid.groupby(['transmitter_id','date','time_of_day'], observed=True, as_index=False)
                      .agg(bin_total=('detections_in_hour','sum'))) #total detections per bin per day
tod_day['date_dt'] = pd.to_datetime(tod_day['date'])


#Initiating empty list to collect results in
anova_results = []

#Looping through mantas, applying labels, running ANOVA test
for manta_label in daily_totals['transmitter_id'].unique():
    label = transmitter_labels.get(manta_label, manta_label)
    sub = tod_day[tod_day['transmitter_id'] == manta_label]
    anova_results.append(oneway_anova_by_bin(sub, label=label)) #appending to list created above

###############################  Manta Agreement on TOD #############################

#Seeing if TOD for individual mantas correspond with each other; initially I was accidentally averaging in TOD bins and checking the significance of that, this should fix that
#Using chi-squared test, which is testing independence between Mantas over averaged bins
#Then, running Spearman test to see if the rise/fall of TOD of each manta match each other
pattern_df = (tod_day.groupby(['transmitter_id', 'time_of_day'], observed=True, as_index=False)['bin_total']
              .mean()
              .pivot(index='time_of_day', columns='transmitter_id', values='bin_total')
              .rename(columns=transmitter_labels)
              .dropna(how='any'))

tod_chi2 = tod_pval = tod_dof = np.nan
tod_s = tod_sval = np.nan

if pattern_df.shape[0] >= 2:
    # Chi-squared test (distribution difference)
    chi_df = pattern_df.round().astype(int)
    tod_chi2, tod_pval, tod_dof, expected = chi2_contingency(chi_df)

    # Spearman correlation (pattern similarity)
    tod_s, tod_sval = spearmanr(pattern_df['Return-Visitor Manta'], pattern_df['Resident Manta'])

    print("\n[Time-of-Day Agreement Tests]")
    print(pattern_df.round(2))
    print(f"  Chi-squared: χ² = {tod_chi2:.2f}, df = {tod_dof}, p = {tod_pval:.3f}")
    print(f"  Spearman correlation: ρ = {tod_s:.3f}, p = {tod_sval:.3f}")

    anova_results.append({
        'Manta': 'TOD Chi-squared test',
        'F-stat': tod_chi2,
        'p-val': tod_pval,
        'eta^2': np.nan
    })

    anova_results.append({
        'Manta': 'TOD Spearman correlation',
        'F-stat': tod_s,
        'p-val': tod_sval,
        'eta^2': np.nan
    })

else:
    print("\n[Time-of-Day Agreement Tests] Not enough bins to compare.")

#Turning list of dictionaries into DataFrame
anova_df = pd.DataFrame(anova_results)

#Saving to excel file with clear labels
anova_output_path = os.path.join(folders["sheets"], "manta_time_of_day_stats.xlsx")
anova_df.to_excel(anova_output_path, sheet_name="time_of_day_ANOVA", index=False)

#Printing concise summary in kernel
print("\nANOVA Summary by Manta:")
print(anova_df[['Manta','F-stat','p-val','eta^2']]) #adding descriptor labals for ease of read


[Time-of-Day Agreement Tests]
transmitter_id     Return-Visitor Manta  Resident Manta
time_of_day                                            
Night (0-6)                       73.11          171.56
Morning (6-12)                    41.57          143.71
Afternoon (12-18)                 43.52          136.18
Evening (18-24)                   53.73          161.15
  Chi-squared: χ² = 3.28, df = 3, p = 0.351
  Spearman correlation: ρ = 0.800, p = 0.200

ANOVA Summary by Manta:
                      Manta     F-stat         p-val     eta^2
0      Return-Visitor Manta  19.309732  4.056471e-12  0.063874
1            Resident Manta  81.115711  3.500408e-50  0.084868
2      TOD Chi-squared test   3.276787  3.508866e-01       NaN
3  TOD Spearman correlation   0.800000  2.000000e-01       NaN


## Step 6: Statistics for Agreement Between Mantas


In [11]:
#########################################################################################################
# This block is to look at statistical agreement between the resident and returning visitor mantas
# Do their increases/decreases in activity co-occur? Using Pearson and Spearman correlations (the
# former is a bit stricter on the linearality of the co-occurence). Pearson tests for linear correlation
# using normally-distributed data, while spearman looks for correlation on non-linear or not normal
# data. This only includes times when they were both there, so it is looking at if hourly behavior is
# related. Could have also been done using all mantas! Next time :) AI used for help in this block for debugging and stat-package jargon assistance.
#########################################################################################################

#Initalizing empty list for results 
agreement_results = []

#Pivot daily_totals to have one column per manta instead of rows per detection
#One row per date, one column per manta, values are detections per day per manta. Ensures sorting by date.
daily_wide = daily_totals.pivot(index='date_dt', columns='transmitter_id', values='total_detections').sort_index()

daily_wide = daily_wide.rename(columns=transmitter_labels) #renaming columns using Manta labels

#Drop dates where one or both mantas had no valid detections -- no comparison if one not detected that day
aligned = daily_wide.dropna(how='any')

if len(aligned) >= 3: #if at least three days of data present, get stats
    #pearson r is linear test, higher = linear movement. spearman p high when busy/slow days correlate, even if not exactly linear.
    #p-values for both tell you how likely that pattern is to be not random.
    hourly_p, hourly_pval = pearsonr(aligned['Return-Visitor Manta'], aligned['Resident Manta'])
    hourly_s, hourly_sval = spearmanr(aligned['Return-Visitor Manta'], aligned['Resident Manta'])


    #printing in kernel
    print("\n[Agreement between mantas: daily totals]") 
    print(f"  Pearson r = {hourly_p:.3f} (p={hourly_pval:.3g})")
    print(f"  Spearman ρ = {hourly_s:.3f} (p={hourly_sval:.3g})")
    
    #Organizing results for output into excel file
    agreement_results.append({
        "Manta_1": "Return-Visitor Manta",
        "Manta_2": "Resident Manta",
        "n_days": len(aligned),
        "pearson_r": hourly_p,
        "pearson_p": hourly_pval,
        "spearman_rho": hourly_s,
        "spearman_p": hourly_sval
    })

else:
    print("\n[Agreement between mantas] Not enough aligned data points for correlation.") #back to beginning of loop, if condition not met then skips to this

#Saving appended list to excel 
agreement_df = pd.DataFrame(agreement_results)

agreement_output_path = os.path.join(folders["sheets"], "manta_agreement_stats.xlsx")
agreement_df.to_excel(agreement_output_path, sheet_name="agreement_stats", index=False)

print(agreement_df)


[Agreement between mantas: daily totals]
  Pearson r = 0.271 (p=2.83e-06)
  Spearman ρ = 0.295 (p=3.09e-07)
                Manta_1         Manta_2  n_days  pearson_r  pearson_p  \
0  Return-Visitor Manta  Resident Manta     291   0.270551   0.000003   

   spearman_rho    spearman_p  
0      0.294606  3.087718e-07  


## Step 7: Interpretation of Agreement and TOD


In [12]:
#########################################################################################################
# I added this block of code so that a text-interpretation of the numbers can be printed for both the TOD
# output and the agreement statistics. Optional, but numbers hard to interpret in kernel format and excel
# files not viewable without download.
#########################################################################################################

#1) ANOVA Results

print("\nInterpretation of ANOVA Results:")

for _, row in anova_df.iterrows():
    manta = row["Manta"]
    f_val = row["F-stat"]
    p_val = row["p-val"]
    eta_sq = row["eta^2"]

    if np.isnan(f_val) or np.isnan(p_val):
        print(f"  • {manta}: Not enough data across time-of-day bins to perform ANOVA.")
        continue

    # Significance interpretation
    if p_val < 0.05:
        sig_text = "significant"
    else:
        sig_text = "not significant"

    # Effect size interpretation
    if eta_sq < 0.01:
        effect = "very small"
    elif eta_sq < 0.06:
        effect = "small"
    elif eta_sq < 0.14:
        effect = "moderate"
    else:
        effect = "large"

    print(f"  • {manta}: Time-of-day effect is {sig_text} (p={p_val:.3f}), "
          f"F={f_val:.2f}, η²={eta_sq:.3f} → {effect} effect size.")
          

#2) Hourly Behavior Agreement Results

if not agreement_df.empty:
    print("\nInterpretation of Hourly Agreement Results:")

    for _, row in agreement_df.iterrows():
        manta_1 = row["Manta_1"]
        manta_2 = row["Manta_2"]
        n_days = int(row["n_days"])
        r_p = row["pearson_r"]
        p_p = row["pearson_p"]
        r_s = row["spearman_rho"]
        p_s = row["spearman_p"]

        #Pearson for hourly 
        if p_p < 0.05:
            pearson_sig = "significant"
        else:
            pearson_sig = "not significant"

        if abs(r_p) < 0.3:
            pearson_strength = "weak"
        elif abs(r_p) < 0.5:
            pearson_strength = "moderate"
        else:
            pearson_strength = "strong"

        #Spearman for hourly 
        if p_s < 0.05:
            spearman_sig = "significant"
        else:
            spearman_sig = "not significant"

        if abs(r_s) < 0.3:
            spearman_strength = "weak"
        elif abs(r_s) < 0.5:
            spearman_strength = "moderate"
        else:
            spearman_strength = "strong"

        # Print concise summary
        print(f"  • Between {manta_1} and {manta_2} ({n_days} overlapping days):")
        print(f"     - Pearson: {pearson_strength} correlation (r={r_p:.3f}, p={p_p:.3f}, {pearson_sig})")
        print(f"     - Spearman: {spearman_strength} correlation (ρ={r_s:.3f}, p={p_s:.3f}, {spearman_sig})")

else:
    print("\nInterpretation of Agreement Results: No valid overlapping data available.")


#3) Time-of-Day (TOD) Chi-Squared Test Results

print("\nInterpretation of Time-of-Day (TOD) Agreement:")

chi_row = anova_df[anova_df["Manta"].str.contains("Chi-squared", case=False, na=False)]
spear_row = anova_df[anova_df["Manta"].str.contains("Spearman", case=False, na=False)]

if not chi_row.empty:
    chi_val = chi_row["F-stat"].iloc[0]
    chi_p = chi_row["p-val"].iloc[0]
    chi_sig = "significant" if chi_p < 0.05 else "not significant"
    print(f"  • Chi-squared: χ²={chi_val:.2f}, p={chi_p:.3f} → {chi_sig}")

if not spear_row.empty:
    s_val = spear_row["F-stat"].iloc[0]
    s_p = spear_row["p-val"].iloc[0]
    s_sig = "significant" if s_p < 0.05 else "not significant"

    if abs(s_val) < 0.3:
        s_strength = "weak"
    elif abs(s_val) < 0.5:
        s_strength = "moderate"
    else:
        s_strength = "strong"

    print(f"  • Spearman correlation: ρ={s_val:.3f}, p={s_p:.3f} → {s_strength}, {s_sig}")

if not chi_row.empty or not spear_row.empty:
    if not chi_row.empty and chi_p >= 0.05 and not spear_row.empty and s_val > 0.5:
        print("     → The two mantas show strongly similar diel (time-of-day) activity patterns, "
              "with no significant difference in distribution.")
    elif not chi_row.empty and chi_p < 0.05:
        print("     → The two mantas differ significantly in their time-of-day activity distributions.")
    else:
        print("     → Their diel activity patterns are similar but not statistically distinct.")
else:
    print("  • No valid TOD test results available.")


Interpretation of ANOVA Results:
  • Return-Visitor Manta: Time-of-day effect is significant (p=0.000), F=19.31, η²=0.064 → moderate effect size.
  • Resident Manta: Time-of-day effect is significant (p=0.000), F=81.12, η²=0.085 → moderate effect size.
  • TOD Chi-squared test: Time-of-day effect is not significant (p=0.351), F=3.28, η²=nan → large effect size.
  • TOD Spearman correlation: Time-of-day effect is not significant (p=0.200), F=0.80, η²=nan → large effect size.

Interpretation of Hourly Agreement Results:
  • Between Return-Visitor Manta and Resident Manta (291 overlapping days):
     - Pearson: weak correlation (r=0.271, p=0.000, significant)
     - Spearman: weak correlation (ρ=0.295, p=0.000, significant)

Interpretation of Time-of-Day (TOD) Agreement:
  • Chi-squared: χ²=3.28, p=0.351 → not significant
  • Spearman correlation: ρ=0.800, p=0.200 → strong, not significant
     → The two mantas show strongly similar diel (time-of-day) activity patterns, with no significa